In [75]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
#shapefile = '/content/drive/MyDrive/Colab Notebooks/2012_Metropolitan_King_County_Council_Districts___kccdst_area_2012.shp'

housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
housing['date'] = pd.to_datetime(housing['date'], format='%Y%m%dT%H%M%S')
start_date = pd.Timestamp('2014-05-01')
housing['days_since_start'] = (housing['date'] - start_date).dt.days
housing.drop('date', axis=1, inplace=True)

In [77]:
import geopandas as gpd
from shapely.geometry import Point

shapefile_path = '/content/drive/MyDrive/Colab Notebooks/Cities_and_Unincorporated_King_County___city_kc_area.shp'
csv_path = '/content/drive/MyDrive/Colab Notebooks/Cities_and_Unincorporated_King_County___city_kc_area.csv'
shapefile_data = gpd.read_file(shapefile_path)
print(shapefile_data.head())
csv_data = gpd.read_file(csv_path)
print(csv_data.head())
shapefile_data["city_name"] = csv_data["CITYNAME"]

#regions = shapefile_data[['city_name', 'geometry']]

def get_region(lat, long):
    point = Point(long, lat)  # Assuming lat and long are the column names in your DataFrame
    for index, region in shapefile_data.iterrows():
        if region['geometry'].contains(point):
            return region['city_name']
    return "Skykomish"

housing['region'] = housing.apply(lambda row: get_region(row['lat'], row['long']), axis=1)

                                            geometry
0  POLYGON ((-122.24022 47.29353, -122.24307 47.2...
1  POLYGON ((-122.22820 47.35395, -122.22546 47.3...
2  POLYGON ((-122.20276 47.58679, -122.20276 47.5...
3  POLYGON ((-121.96191 47.28836, -121.96191 47.2...
4  POLYGON ((-122.04854 47.34368, -122.04412 47.3...
  OBJECTID JURIS       CITYNAME      SHAPE_Length        SHAPE_Area geometry
0        1    AL         Algona  38144.6671787663    36411045.81911     None
1        2    AU         Auburn  208679.325076434  785915572.159009     None
2        3    BA     Beaux Arts  6720.78580239548  2274706.65404921     None
3        4    BD  Black Diamond  6629.56958821086   2412696.7229177     None
4        5    BD  Black Diamond  85786.1071157801  198366063.758258     None


In [78]:
print(housing['region'].value_counts())
print(housing['region'].isna().sum())

Seattle             6241
King County         2722
Bellevue            1218
Renton               900
Sammamish            892
Kirkland             881
Kent                 772
Federal Way          674
Shoreline            574
Auburn               505
Redmond              480
Maple Valley         461
Issaquah             410
Burien               409
Kenmore              267
Snoqualmie           267
Mercer Island        253
Des Moines           249
Covington            242
Bothell              195
SeaTac               166
Lake Forest Park     166
Enumclaw             145
Duvall               134
Woodinville          129
Newcastle            128
Tukwila               91
Normandy Park         81
North Bend            74
Black Diamond         50
Clyde Hill            47
Medina                46
Pacific               38
Carnation             24
Algona                24
Yarrow Point          17
Milton                16
Beaux Arts             5
Skykomish              4
Hunts Point            3


In [79]:
adjustment_percentages = {
    "Seattle": 13.9,
    "King County": 11.7,
    "Bellevue": 18.2,
    "Renton": 15.0,
    "Sammamish": 15.8,
    "Kirkland": 15.6,
    "Kent": 13.5,
    "Federal Way": 13.2,
    "Shoreline": 12.9,
    "Auburn": 11.7,
    "Redmond": 17.8,
    "Maple Valley": 14.3,
    "Issaquah": 14.7,
    "Burien": 13.8,
    "Kenmore": 11.5,
    "Snoqualmie": 11.5,
    "Mercer Island": 16.4,
    "Des Moines": 12.7,
    "Covington": 12.5,
    "Bothell": 14.0,
    "SeaTac": 12.5,
    "Lake Forest Park": 12.5,
    "Enumclaw": 10.9,
    "Duvall": 11.0,
    "Woodinville": 12.2,
    "Newcastle": 12.1,
    "Tukwila": 11.0,
    "Normandy Park": 10.1,
    "North Bend": 10.0,
    "Black Diamond": 8.9,
    "Clyde Hill": 9.7,
    "Medina": 9.6,
    "Pacific": 8.7,
    "Carnation": 6.7,
    "Algona": 6.7,
    "Yarrow Point": 5.2,
    "Milton": 5.1,
    "Beaux Arts": 2.0,
    "Hunts Point": 1.0,
    "Skykomish": 1.0
}


def calculate_adjusted_price(row):
    region = row['region'] 
    price = row['price'] 
    days_diff = row['days_since_start']  # Extract relavant information
    
    # Get the adjustment percentage for the region
    adjustment_percentage = adjustment_percentages.get(region)

    # Calculate the adjustment based on the percentage and the number of days
    try:
        adjustment = (adjustment_percentage / 365) * days_diff
    except TypeError:
        return price

    # Calculate the adjusted price
    adjusted_price = price - (price * (adjustment / 100))

    return adjusted_price

# It's not pretty, but at least I was able to have an AI write it for me. Deals with one-hot encoding.
def reverse_adjusted_price(adjusted_price, row):
    if row['region_Seattle'] == 1:
        region = "Seattle"
    elif row['region_King County'] == 1:
        region = "King County"
    elif row['region_Bellevue'] == 1:
        region = "Bellevue"
    elif row['region_Renton'] == 1:
        region = "Renton"
    elif row['region_Sammamish'] == 1:
        region = "Sammamish"
    elif row['region_Kirkland'] == 1:
        region = "Kirkland"
    elif row['region_Kent'] == 1:
        region = "Kent"
    elif row['region_Federal Way'] == 1:
        region = "Federal Way"
    elif row['region_Shoreline'] == 1:
        region = "Shoreline"
    elif row['region_Auburn'] == 1:
        region = "Auburn"
    elif row['region_Redmond'] == 1:
        region = "Redmond"
    elif row['region_Maple Valley'] == 1:
        region = "Maple Valley"
    elif row['region_Issaquah'] == 1:
        region = "Issaquah"
    elif row['region_Burien'] == 1:
        region = "Burien"
    elif row['region_Kenmore'] == 1:
        region = "Kenmore"
    elif row['region_Snoqualmie'] == 1:
        region = "Snoqualmie"
    elif row['region_Mercer Island'] == 1:
        region = "Mercer Island"
    elif row['region_Des Moines'] == 1:
        region = "Des Moines"
    elif row['region_Covington'] == 1:
        region = "Covington"
    elif row['region_Bothell'] == 1:
        region = "Bothell"
    elif row['region_SeaTac'] == 1:
        region = "SeaTac"
    elif row['region_Lake Forest Park'] == 1:
        region = "Lake Forest Park"
    elif row['region_Enumclaw'] == 1:
        region = "Enumclaw"
    elif row['region_Duvall'] == 1:
        region = "Duvall"
    elif row['region_Woodinville'] == 1:
        region = "Woodinville"
    elif row['region_Newcastle'] == 1:
        region = "Newcastle"
    elif row['region_Tukwila'] == 1:
        region = "Tukwila"
    elif row['region_Normandy Park'] == 1:
        region = "Normandy Park"
    elif row['region_North Bend'] == 1:
        region = "North Bend"
    elif row['region_Black Diamond'] == 1:
        region = "Black Diamond"
    elif row['region_Clyde Hill'] == 1:
        region = "Clyde Hill"
    elif row['region_Medina'] == 1:
        region = "Medina"
    elif row['region_Pacific'] == 1:
        region = "Pacific"
    elif row['region_Carnation'] == 1:
        region = "Carnation"
    elif row['region_Algona'] == 1:
        region = "Algona"
    elif row['region_Yarrow Point'] == 1:
        region = "Yarrow Point"
    elif row['region_Milton'] == 1:
        region = "Milton"
    elif row['region_Beaux Arts'] == 1:
        region = "Beaux Arts"
    elif row['region_Hunts Point'] == 1:
        region = "Hunts Point"
    elif row['region_Skykomish'] == 1:
        region = "Skykomish"
    else:
        region = None
    
    if region is None:
        # No active region column found, return adjusted price as is
        return adjusted_price

    adjustment_percentage = adjustment_percentages.get(region)
    days_diff = row['days_since_start']  # Extract relavant information
    try:
        adjustment = (adjustment_percentage / 365) * days_diff
    except TypeError:
        adjustment = 0
    price = adjusted_price + (adjusted_price * (adjustment / 100))
    return price

# Apply the calculate_adjusted_price function to create the 'price_adjusted' column
housing['price_adjusted'] = housing.apply(calculate_adjusted_price, axis=1)
housing.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,days_since_start,region,price_adjusted
0,1565930130,4,3.25,3760,4675,2.0,0,0,3,8,...,0,98038,47.3862,-122.048,3280,4033,429900.0,187,Maple Valley,398404.230411
1,3279000420,3,1.75,1460,7800,1.0,0,0,2,7,...,0,98023,47.3035,-122.382,1310,7865,233000.0,259,Federal Way,211175.879452
2,194000575,4,1.00,1340,5800,1.5,0,2,3,7,...,0,98116,47.5658,-122.389,1900,5800,455000.0,166,Seattle,426236.520548
3,2115510160,3,1.75,1440,8050,1.0,0,0,3,8,...,0,98023,47.3187,-122.390,1790,7488,258950.0,221,Federal Way,238253.864658
4,7522500005,2,1.50,1780,4750,1.0,0,0,4,7,...,0,98117,47.6859,-122.395,1690,5962,555000.0,106,Seattle,532596.246575


In [80]:
def ensure_columns(df, column_names):
    missing_columns = [column for column in column_names if column not in df.columns]
    for column in missing_columns:
        df[column] = 0
    return df

In [92]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor
keys_list = list(adjustment_percentages.keys())
column_names = ['region_' + item for item in keys_list]
housing_encoded = housing.copy()
column_names += housing_encoded.columns.tolist()
column_names.remove('price')
column_names.remove('price_adjusted')
column_names.remove('region')
# Split the data into train, validation, and test sets
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(
    housing_encoded.drop(['price', 'price_adjusted'], axis=1),
    housing_encoded[['price', 'price_adjusted']],  # Include both "price" and "price_adjusted" columns
    test_size=0.2,
    random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val,
    y_train_val,
    test_size=0.125,  # 10% of the original dataset size (0.1 / 0.8)
    random_state=42
)

X_set = X_train
X_set[['price_adjusted', 'price']] = y_train[['price_adjusted', 'price']]
y_set = X_set['region']
X_set.drop(['region'], axis=1, inplace=True)
from imblearn.over_sampling import RandomOverSampler
oversampler = RandomOverSampler()
X_resampled, y_resampled = oversampler.fit_resample(X_set, y_set)
X_resampled['region'] = y_resampled
X_train = X_resampled
y_train = X_train[['price_adjusted', 'price']]
X_train.drop(['price_adjusted', 'price'], axis=1, inplace=True)

# Perform one-hot encoding on the categorical features
X_train = pd.get_dummies(X_train)
X_train = ensure_columns(X_train, column_names)
X_val = pd.get_dummies(X_val)
X_val = ensure_columns(X_val, column_names)
X_test_val = pd.get_dummies(X_test_val)
X_test_val = ensure_columns(X_test_val, column_names)
X_train = X_train.sort_index(axis=1)
X_val = X_val.sort_index(axis=1)
X_test_val = X_test_val.sort_index(axis=1)

# Separate the price and price_adjusted columns in the train, validation, and test sets
y_train_price = y_train['price']
y_train_adjusted = y_train['price_adjusted']
y_val_price = y_val['price']
y_val_adjusted = y_val['price_adjusted']
y_test_price = y_test_val['price']
y_test_adjusted = y_test_val['price_adjusted']

# Create an XGBRegressor model
model = XGBRegressor(n_estimators=1000, max_depth=5,learning_rate=0.05,subsample=.5)

# Train the model on the training data
model.fit(X_train, y_train_adjusted)

# Make predictions on the validation set
y_pred_val_adjusted = model.predict(X_val)

# Calculate the R^2 score on the validation set using "price_adjusted"
r2_val_adjusted = r2_score(y_val_adjusted, y_pred_val_adjusted)
print(f"R^2 score on validation set (price_adjusted): {r2_val_adjusted}")

# Make predictions on the test set
y_pred_test_adjusted = model.predict(X_test_val)

# Add y_pred_test_adjusted as a column to X_test_val
X_test_val['predicted_price_adjusted'] = y_pred_test_adjusted

# Apply the reverse normalization function using a lambda function
X_test_val['predicted_price'] = X_test_val.apply(lambda row: reverse_adjusted_price(row['predicted_price_adjusted'], row), axis=1)

# Extract the predicted prices from X_test_val
y_pred_test = X_test_val['predicted_price']

# Calculate the R^2 score on the test set using the actual "price" column
r2_test = r2_score(y_test_price, y_pred_test)
print(f"R^2 score on test set (price): {r2_test}")

# Calculate the root mean squared error (RMSE) on the test set using the actual "price" column
rmse_test = mean_squared_error(y_test_price, y_pred_test, squared=False)
print(f"RMSE on test set (price): {rmse_test}")


R^2 score on validation set (price_adjusted): 0.9043809930446189
R^2 score on test set (price): 0.8771112468646366
RMSE on test set (price): 136139.19675051523
